In [19]:
import os
import sys
import boto3
from boto3.s3.transfer import TransferConfig
import json
import geopandas as gpd
from io import BytesIO

config = TransferConfig(use_threads=False)

def start_session():
    """
    Start a new boto3 session

    Returns:
        session: boto3 session
    """
    session = boto3.Session()

    return session

def get_mfa_serial(session):
    """
    Get the multi-factor authentication (MFA) serial number from the session
    
    Args:
        session: boto3 session

    Returns:
        mfa_serial: MFA serial number
    """
    mfa_serial = session._session.full_config['profiles']['default']['mfa_serial']

    return mfa_serial

def get_mfa_token():
    """
    Get the MFA token from the user
    
    Returns:
        mfa_token: MFA token
    """
    mfa_token = input('Please enter your 6 digit MFA code:')
    
    return mfa_token

def get_session_token(session, mfa_serial=None, mfa_token=None, mfa_required=True):
    """
    Get a session token from AWS Security Token Service (STS)
    
    Args:
        session: boto3 session
        mfa_serial: MFA serial number
        mfa_token: MFA token
    
    Returns:
        MFA_validated_token: MFA validated token
    """
    sts = session.client('sts')

    if mfa_required:
        return sts.get_session_token(SerialNumber=mfa_serial, TokenCode=mfa_token)
    
    else:
        return sts.get_session_token()

def decompose_token(token):
    """
    Decompose the token into its components

    Args:
        token: MFA validated token

    Returns:
        session_token: Session token from AWS STS
        secret_access_key: Secret access key from AWS STS
        access_key_id: Access key id from AWS STS
    """
    credentials = token.get('Credentials', {})
    session_token = credentials.get('SessionToken')
    secret_access_key = credentials.get('SecretAccessKey')
    access_key_id = credentials.get('AccessKeyId')

    return session_token, secret_access_key, access_key_id

# (1) Connect to boto3 and create S3 client

In [20]:
# Create a boto3 session
session = start_session()

# Handle SMCE MFA requirement
mfa_serial = get_mfa_serial(session=session)
mfa_token = get_mfa_token()
MFA_validated_token = get_session_token(session=session, mfa_serial=mfa_serial, mfa_token=mfa_token)
session_token, secret_access_key, access_key_id = decompose_token(MFA_validated_token)

# Create an S3 client
s3 = session.client('s3', 
            aws_session_token=session_token,
            aws_secret_access_key=secret_access_key,
            aws_access_key_id=access_key_id
            )

# (2) List all S3 buckets

In [21]:
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  729474902545-smce-hpcscript
  config-bucket-smce-729474902545
  cur-reports-smce-729474902545
  smce-ansible-729474902545
  snow-observing-systems


# (3) List all files

In [22]:
response = s3.list_objects_v2(Bucket='snow-observing-systems')

# Check if the bucket contains any objects
if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("Bucket is empty or does not exist.")

Optimization_result.geojson
WBD_10_HU2_4326.geojson
s3_update.ipynb


# (4) Read contents of a file

In [23]:
# Get the object from the bucket
response = s3.get_object(Bucket='snow-observing-systems', Key='Optimization_result.geojson')

# Read the contents of the file
file_contents = response['Body'].read()

# Parse the contents as GeoJSON
geojson_data = gpd.read_file(BytesIO(file_contents))

# (5) Download a file

In [24]:
s3.download_file('snow-observing-systems', 'Optimization_result.geojson', 'Optimization_result.geojson', Config=config)

# (6) Upload a file

In [25]:
s3.upload_file(
    'Optimization_result.geojson', 'snow-observing-systems', 'Optimization_result.geojson', Config=config
)

# (7) SQS

In [26]:
# Create an S3 client
sqs = session.client('sqs', 
            aws_session_token=session_token,
            aws_secret_access_key=secret_access_key,
            aws_access_key_id=access_key_id
            )

In [27]:
response = sqs.receive_message(
    QueueUrl='https://sqs.us-east-1.amazonaws.com/729474902545/SnowObservingSystemsDataUpdates'
)

In [28]:
response

{'ResponseMetadata': {'RequestId': '3727cfe9-1f04-5f79-afbe-9012a92fb60e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3727cfe9-1f04-5f79-afbe-9012a92fb60e',
   'date': 'Fri, 10 Jan 2025 23:57:31 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}